In [1]:
# Imports and setups

import os
import pyspark
import pyspark.sql.functions as f

conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 14:28:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [2]:
!du -hs .

du: cannot read directory './groupshare': Permission denied
20G	.


In [2]:
#Read CSVs
df = spark.read.option("inferSchema", "true")\
            .option("header", "false")\
            .csv("ProjectData")\
            .toDF('video_id','title','publishedAt','channelId','channelTitle','categoryId','trending_date','tags','view_count','likes','dislikes','comment_count','thumbnail_link','comments_disabled','ratings_disabled')

In [3]:
# df.dtypes

In [4]:
#Create Category Dict
cat_dict = {}

df_cat = spark.read\
            .option('multiLine', True)\
            .json("ProjectData/IN_category_id.json")

df_cat2 = df_cat.select('items').first()

lst = df_cat2.asDict()['items']

for ele in lst:
    ele = ele.asDict()
    key = ele['id']
    ele2 = ele['snippet'].asDict()
    value = ele2['title']
    cat_dict[key] = value

In [5]:
#Create Category Dict
df_cat = spark.read\
            .option('multiLine', True)\
            .json("ProjectData/US_category_id.json")

df_cat2 = df_cat.select('items').first()

lst = df_cat2.asDict()['items']

for ele in lst:
    ele = ele.asDict()
    key = ele['id']
    ele2 = ele['snippet'].asDict()
    value = ele2['title']
    cat_dict[key] = value

In [6]:
#Clean DF
df_clean = df.select('title','publishedAt','channelTitle','categoryId','trending_date','tags','view_count','likes','dislikes','comment_count','comments_disabled','ratings_disabled')
df_clean = df_clean.na.drop()
df = df_clean

In [7]:
#Remove First Row
df=df.filter(df['title'] != 'title')

In [8]:
#Replace CategoryID with Category
from pyspark.sql.functions import udf, col

def replace_cat(id):
    return cat_dict[str(id)]

replace_cat_udf = udf(replace_cat)

df2 = replace_cat_udf(df['categoryId'])

df = df.withColumn('category', df2)
df = df.drop('categoryId')

In [9]:
from pyspark.sql.types import IntegerType

#Convert string timestamp values
df1 = df.withColumn('publishedAt', f.to_timestamp('publishedAt'))

df1 = df1.withColumn('trending_date', f.to_timestamp('publishedAt'))

#Convert to integer type
df1 = df1.withColumn("view_count", df1["view_count"].cast(IntegerType()))
df1 = df1.withColumn("likes", df1["likes"].cast(IntegerType()))
df1 = df1.withColumn("dislikes", df1["dislikes"].cast(IntegerType()))
df1 = df1.withColumn("comment_count", df1["comment_count"].cast(IntegerType()))

#Remove leading whitespaces from Strings
df1 = df1.withColumn("category", f.trim(df.category))

## Top 5 View Counts, Likes, Dislikes by Channel Title

In [10]:
#Aggregate
df2_title = df1.groupBy('channelTitle').agg(f.sum("view_count").alias("sum_view_count"),f.sum("likes").alias("sum_likes"),f.sum("dislikes").alias("sum_dislikes"))
df2_title

channelTitle,sum_view_count,sum_likes,sum_dislikes
SAVANNAH DEXTER,18743801,1163029,71288
Jordan Orme,9156379,717286,1009
The New York Times,56903233,896381,75459
Marcus House,4461265,186765,3860
SPORTSNET,24991198,131983,7199
NBC,60119776,1098729,37335
Lil Nas X,252321376,16874620,626458
Fenty Beauty By R...,3747232,142083,2211
Chris Ramsay,41276247,3607906,7784
By Post,27063509,549553,17515


In [11]:
#Top 5 View Counts
df_maxViewCounts_title = df2_title.select(col('channelTitle'), col('sum_view_count')).sort(f.desc('sum_view_count')).limit(5)
df_maxViewCounts_title


# #
# df_maxViewCounts_title_stats = df_maxViewCounts_title.toPandas()
# df_maxViewCounts_title_stats.to_csv('df_maxViewCounts_title_stats.csv')

channelTitle,sum_view_count
BLACKPINK,17750505277
MrBeast,14571786863
T-Series,7800090285
Big Hit Labels,7393869309
HYBE LABELS,7134963778


In [12]:
# start temp

In [26]:
df_trend = df1.filter((df1['channelTitle'] == 'BLACKPINK'))
df_trend = df_trend.select("channelTitle","publishedAt","view_count").orderBy("publishedAt")

# #
# df_trend_line = df_trend.toPandas()
# df_trend_line.to_csv('df_trend_line.csv')

In [13]:
# end temp

In [11]:
#Top 5 Likes
df_likes_title = df2_title.select(col('channelTitle'), col('sum_likes')).sort(f.desc('sum_likes')).limit(5)
df_likes_title

# #
# df_likes_title_stats = df_likes_title.toPandas()
# df_likes_title_stats.to_csv('df_likes_title_stats.csv')

channelTitle,sum_likes
BLACKPINK,1405411843
MrBeast,980832852
BANGTANTV,816410212
HYBE LABELS,806770326
Big Hit Labels,737933401


In [12]:
#Top 5 Dislikes
df_dislikes_title = df2_title.select(col('channelTitle'), col('sum_dislikes')).sort(f.desc('sum_dislikes')).limit(5)
df_dislikes_title

# #
# df_dislikes_title_stats = df_dislikes_title.toPandas()
# df_dislikes_title_stats.to_csv('df_dislikes_title_stats.csv')

channelTitle,sum_dislikes
FoxStarHindi,114154505
BLACKPINK,21567184
Desi Music Factory,17735645
Big Hit Labels,15925879
T-Series,13592859


## Top 5 View Counts, Likes, Dislikes by Channel Category

In [13]:
#Aggregate
df_cat = df1.groupBy('category').agg(f.sum("view_count").alias("sum_view_count"),f.sum("likes").alias("sum_likes"),f.sum("dislikes").alias("sum_dislikes"))
df_cat

# #
# df_cat_stats = df_cat.toPandas()
# df_cat_stats.to_csv('df_cat_stats.csv')

category,sum_view_count,sum_likes,sum_dislikes
Education,8591999011,471200288,6660300
Gaming,63154944233,3518138775,44644961
Entertainment,151889050247,6975555147,249112583
Travel & Events,1133458368,67808606,975427
Science & Technology,21256617085,943527769,24681285
Sports,41806056655,1090879590,21125712
Howto & Style,8507909870,416061489,12929751
Nonprofits & Acti...,283980702,12992942,117360
Film & Animation,18515009066,781761072,22813383
People & Blogs,36527786274,2034389354,48953660


In [14]:
#Top 5 View Counts
df_maxViewCounts_cat = df_cat.select(col('category'), col('sum_view_count')).sort(f.desc('sum_view_count')).limit(5)
df_maxViewCounts_cat

category,sum_view_count
Music,185778221518
Entertainment,151889050247
Gaming,63154944233
Sports,41806056655
People & Blogs,36527786274


In [15]:
#Top 5 Likes
df_likes_cat = df_cat.select(col('category'), col('sum_likes')).sort(f.desc('sum_likes')).limit(5)
df_likes_cat

category,sum_likes
Music,11648543902
Entertainment,6975555147
Gaming,3518138775
People & Blogs,2034389354
Comedy,1957738854


In [16]:
#Top 5 Dislikes
df_dislikes_cat = df_cat.select(col('category'), col('sum_dislikes')).sort(f.desc('sum_dislikes')).limit(5)
df_dislikes_cat

category,sum_dislikes
Entertainment,249112583
Music,226906062
People & Blogs,48953660
Gaming,44644961
Comedy,30614974


## Bottom 5 View Counts, Likes, Dislikes by Channel Category

In [17]:
# Bottom 5 View Counts
df_minViewCounts_cat = df_cat.select(col('category'), col('sum_view_count')).sort('sum_view_count').limit(5)
df_minViewCounts_cat

category,sum_view_count
Nonprofits & Acti...,283980702
Pets & Animals,902164046
Travel & Events,1133458368
Autos & Vehicles,3005499981
Howto & Style,8507909870


In [18]:
# Bottom 5 Likes
df_min_likes_cat = df_cat.select(col('category'), col('sum_likes')).sort('sum_likes').limit(5)
df_min_likes_cat

category,sum_likes
Nonprofits & Acti...,12992942
Pets & Animals,41987498
Travel & Events,67808606
Autos & Vehicles,137287050
News & Politics,232619920


In [19]:
# Bottom 5 Dislikes
df_min_dislikes_cat = df_cat.select(col('category'), col('sum_dislikes')).sort('sum_dislikes').limit(5)
df_min_dislikes_cat

category,sum_dislikes
Nonprofits & Acti...,117360
Pets & Animals,399182
Travel & Events,975427
Autos & Vehicles,1624543
Education,6660300


## Which category of videos have comments disabled ?

In [20]:
df1_comment_disabled = df1.filter((df1['comments_disabled'] == 'TRUE'))
df1_comment_disabled

title,publishedAt,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category
FIRST DAY IN THE ...,2020-08-11 09:00:29,Ellie and Jared,2020-08-11 09:00:29,family life|life ...,208173,8810,215,0,TRUE,FALSE,People & Blogs
AN UMBRELLA ACADE...,2020-08-08 15:00:11,AidanRGallagher,2020-08-08 15:00:11,[None],1183833,127487,1588,0,TRUE,FALSE,Entertainment
LaBrant Family Ba...,2020-08-08 13:26:41,The LaBrant Fam,2020-08-08 13:26:41,cole and sav|the ...,5849640,287811,5040,0,TRUE,FALSE,People & Blogs
First Day of Scho...,2020-08-11 17:06:52,LexiVee03,2020-08-11 17:06:52,lexivee03|veehive...,260195,17234,213,0,TRUE,FALSE,Howto & Style
FIRST DAY IN THE ...,2020-08-11 09:00:29,Ellie and Jared,2020-08-11 09:00:29,family life|life ...,238369,9701,233,0,TRUE,FALSE,People & Blogs
AN UMBRELLA ACADE...,2020-08-08 15:00:11,AidanRGallagher,2020-08-08 15:00:11,[None],1339532,140406,1733,0,TRUE,FALSE,Entertainment
LaBrant Family Ba...,2020-08-08 13:26:41,The LaBrant Fam,2020-08-08 13:26:41,cole and sav|the ...,6072155,293606,5196,0,TRUE,FALSE,People & Blogs
First Day of Scho...,2020-08-11 17:06:52,LexiVee03,2020-08-11 17:06:52,lexivee03|veehive...,300847,19104,233,0,TRUE,FALSE,Howto & Style
FIRST DAY IN THE ...,2020-08-11 09:00:29,Ellie and Jared,2020-08-11 09:00:29,family life|life ...,268852,10564,247,0,TRUE,FALSE,People & Blogs
AN UMBRELLA ACADE...,2020-08-08 15:00:11,AidanRGallagher,2020-08-08 15:00:11,[None],1472373,151676,1872,0,TRUE,FALSE,Entertainment


In [21]:
df_comment_disabled = df1_comment_disabled.select(col('category')).groupBy('category').count().sort(f.desc('count')).withColumnRenamed('count','comments_disabled_count')
df_comment_disabled

# #
# df_comment_disabled_stats = df_comment_disabled.toPandas()
# df_comment_disabled_stats.to_csv('df_comment_disabled_stats.csv')

category,comments_disabled_count
News & Politics,838
Entertainment,598
Science & Technology,528
People & Blogs,405
Gaming,307
Music,101
Film & Animation,88
Sports,65
Howto & Style,47
Education,41


## Which category of videos have ratings disabled ?

In [22]:
df1_ratings_disabled = df1.filter((df1['ratings_disabled'] == 'TRUE'))
df1_ratings_disabled

title,publishedAt,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category
The Paris Hilton ...,2020-08-17 12:00:13,Paris Hilton,2020-08-17 12:00:13,Paris Hilton|Pari...,330188,0,0,1362,FALSE,TRUE,Entertainment
What Really Happe...,2020-08-16 01:30:01,kenanK TV,2020-08-16 01:30:01,sean o malley|sea...,391246,0,0,969,FALSE,TRUE,Sports
SuperM 슈퍼엠 '100' ...,2020-08-18 11:00:17,SuperM,2020-08-18 11:00:17,[None],958787,0,0,8314,FALSE,TRUE,Music
The Paris Hilton ...,2020-08-17 12:00:13,Paris Hilton,2020-08-17 12:00:13,Paris Hilton|Pari...,600752,0,0,1898,FALSE,TRUE,Entertainment
What Really Happe...,2020-08-16 01:30:01,kenanK TV,2020-08-16 01:30:01,sean o malley|sea...,396636,0,0,947,FALSE,TRUE,Sports
SuperM 슈퍼엠 '100' ...,2020-08-18 11:00:17,SuperM,2020-08-18 11:00:17,[None],1122081,0,0,8797,FALSE,TRUE,Music
The Paris Hilton ...,2020-08-17 12:00:13,Paris Hilton,2020-08-17 12:00:13,Paris Hilton|Pari...,737888,0,0,2026,FALSE,TRUE,Entertainment
What Really Happe...,2020-08-16 01:30:01,kenanK TV,2020-08-16 01:30:01,sean o malley|sea...,399271,0,0,949,FALSE,TRUE,Sports
SuperM 슈퍼엠 '100' ...,2020-08-18 11:00:17,SuperM,2020-08-18 11:00:17,[None],1213461,0,0,9025,FALSE,TRUE,Music
The Paris Hilton ...,2020-08-17 12:00:13,Paris Hilton,2020-08-17 12:00:13,Paris Hilton|Pari...,887734,0,0,2214,FALSE,TRUE,Entertainment


In [23]:
df_ratings_disabled = df1_ratings_disabled.select(col('category')).groupBy('category').count().sort(f.desc('count')).withColumnRenamed('count','ratings_disabled_count')
df_ratings_disabled

# #
# df_ratings_disabled_stats = df_ratings_disabled.toPandas()
# df_ratings_disabled_stats.to_csv('df_ratings_disabled_stats.csv')

category,ratings_disabled_count
News & Politics,269
People & Blogs,242
Gaming,208
Music,187
Entertainment,170
Sports,129
Film & Animation,95
Science & Technology,40
Howto & Style,39
Education,30


## Most watched video in India and USA overall

In [24]:
# Top 5 watched video
df_maxview = df1.sort(f.desc('view_count')).limit(5)
df_maxview

title,publishedAt,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,277791741,12993894,0,3534337,FALSE,FALSE,Music
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,273162966,12937252,0,3516745,FALSE,FALSE,Music
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,268758295,12882841,0,3504692,FALSE,FALSE,Music
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,264459017,12829059,0,3491132,FALSE,FALSE,Music
BTS (방탄소년단) 'Butt...,2021-05-20 23:46:13,HYBE LABELS,2021-05-20 23:46:13,BIGHIT|빅히트|방탄소년단|...,264407389,16021534,150989,6738537,FALSE,FALSE,Music


## Most watched video in India and USA Year wise

In [25]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
 
df_maxview_year = df1.withColumn('publishedAt_Year',year(df1['publishedAt']))

In [26]:
# Check distinct years
df_maxview_year.select('publishedAt_Year').groupBy('publishedAt_Year').count()

publishedAt_Year,count
2022,66805
2020,55344
2021,102212


In [27]:
# Top 5 watched video in 2020
df_maxview_year_20 = df_maxview_year.filter((df_maxview_year['publishedAt_Year'] == '2020'))
df_maxview_year_20 = df_maxview_year_20.sort(f.desc('view_count')).limit(5)
df_maxview_year_20

title,publishedAt,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category,publishedAt_Year
BTS (방탄소년단) 'Dyna...,2020-08-20 23:58:10,Big Hit Labels,2020-08-20 23:58:10,BIGHIT|빅히트|방탄소년단|...,253995993,16115240,756696,6249727,FALSE,FALSE,Music,2020
BTS (방탄소년단) 'Dyna...,2020-08-20 23:58:10,Big Hit Labels,2020-08-20 23:58:10,BIGHIT|빅히트|방탄소년단|...,244507902,15948359,738811,6188781,FALSE,FALSE,Music,2020
BTS (방탄소년단) 'Dyna...,2020-08-20 23:58:10,Big Hit Labels,2020-08-20 23:58:10,BIGHIT|빅히트|방탄소년단|...,232649205,15735551,714196,6064883,FALSE,FALSE,Music,2020
BTS (방탄소년단) 'Dyna...,2020-08-20 23:58:10,Big Hit Labels,2020-08-20 23:58:10,BIGHIT|빅히트|방탄소년단|...,232649205,15735533,714194,6065230,FALSE,FALSE,Music,2020
BTS (방탄소년단) 'Dyna...,2020-08-20 23:58:10,Big Hit Labels,2020-08-20 23:58:10,BIGHIT|빅히트|방탄소년단|...,219110491,15460834,674291,5810606,FALSE,FALSE,Music,2020


In [28]:
# Top 5 watched video in 2021
df_maxview_year_21 = df_maxview_year.filter((df_maxview_year['publishedAt_Year'] == '2021'))
df_maxview_year_21 = df_maxview_year_21.sort(f.desc('view_count')).limit(5)
df_maxview_year_21

title,publishedAt,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category,publishedAt_Year
BTS (방탄소년단) 'Butt...,2021-05-20 23:46:13,HYBE LABELS,2021-05-20 23:46:13,BIGHIT|빅히트|방탄소년단|...,264407389,16021534,150989,6738537,FALSE,FALSE,Music,2021
BTS (방탄소년단) 'Butt...,2021-05-20 23:46:13,HYBE LABELS,2021-05-20 23:46:13,BIGHIT|빅히트|방탄소년단|...,264407389,16021542,150990,6738565,FALSE,FALSE,Music,2021
Turn into orbeez ...,2021-07-03 00:04:57,FFUNTV,2021-07-03 00:04:57,[None],206202284,6840430,240769,2826,FALSE,FALSE,Entertainment,2021
Turn into orbeez ...,2021-07-03 00:04:57,FFUNTV,2021-07-03 00:04:57,[None],204846218,6791654,237038,2792,FALSE,FALSE,Entertainment,2021
Turn into orbeez ...,2021-07-03 00:04:57,FFUNTV,2021-07-03 00:04:57,[None],203532581,6745231,233458,2765,FALSE,FALSE,Entertainment,2021


In [29]:
# Top 5 watched video in 2022
df_maxview_year_22 = df_maxview_year.filter((df_maxview_year['publishedAt_Year'] == '2022'))
df_maxview_year_22 = df_maxview_year_22.sort(f.desc('view_count')).limit(5)
df_maxview_year_22

title,publishedAt,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category,publishedAt_Year
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,277791741,12993894,0,3534337,FALSE,FALSE,Music,2022
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,273162966,12937252,0,3516745,FALSE,FALSE,Music,2022
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,268758295,12882841,0,3504692,FALSE,FALSE,Music,2022
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,264459017,12829059,0,3491132,FALSE,FALSE,Music,2022
BLACKPINK - ‘Pink...,2022-08-19 00:00:13,BLACKPINK,2022-08-19 00:00:13,YG Entertainment|...,260126694,12773474,0,3479717,FALSE,FALSE,Music,2022


## Most watched video in India and USA Category wise

In [30]:
# Top watched videos category wise
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
windowDept = Window.partitionBy("category").orderBy(col("view_count").desc())
df_maxview_cat = df1.withColumn("row",row_number().over(windowDept)).filter(col("row") == 1).drop("row")
df_maxview_cat


# #
# df_maxview_cat_stats = df_maxview_cat.toPandas()
# df_maxview_cat_stats.to_csv('df_maxview_cat_stats.csv')


# Alternate approach
# category_list = df1.select('category').distinct().rdd.flatMap(lambda x: x).collect()
# df_max_cat = df1.filter((df1['category'] == 'Blank'))
# for cat in category_list: 
#     df1_t = df1.filter((df1['category'] == cat))
#     df1_t = df1_t.sort(f.desc('view_count')).limit(1)
#     df_max_cat = df_max_cat.union(df1_t)
    
# df_max_cat

title,publishedAt,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category
jai shree ram 🚩#...,2022-03-14 23:21:02,CHANDAN ART ACADEMY,2022-03-14 23:21:02,[None],149615603,7940311,0,48945,FALSE,FALSE,Education
"AMONG US, but wit...",2020-10-08 20:16:14,The Pixel Kingdom,2020-10-08 20:16:14,among us|100 play...,73728043,2337792,55198,69154,FALSE,FALSE,Gaming
Turn into orbeez ...,2021-07-03 00:04:57,FFUNTV,2021-07-03 00:04:57,[None],206202284,6840430,240769,2826,FALSE,FALSE,Entertainment
Crazy #alluarjun ...,2021-12-08 08:16:02,Dr.Harrsha Artist,2021-12-08 08:16:02,[None],86415224,5676872,0,25975,FALSE,FALSE,Film & Animation
HOW TO GO THROUGH...,2021-05-15 14:29:43,5-Minute Crafts F...,2021-05-15 14:29:43,5-Minute Crafts|5...,89075984,2293772,97541,13435,FALSE,FALSE,Howto & Style
Denzel Washington...,2022-04-03 10:58:54,T.D. Jakes,2022-04-03 10:58:54,denzel washington...,6045361,71043,0,19442,FALSE,FALSE,Nonprofits & Acti...
Amazon’s Big Game...,2021-02-02 08:25:20,amazon,2021-02-02 08:25:20,[None],77745621,51199,5779,7584,FALSE,FALSE,Science & Technology
Dice Stacks from ...,2021-10-06 17:44:27,That's Amazing Sh...,2021-10-06 17:44:27,[None],103564168,7174425,120961,23727,FALSE,FALSE,Sports
Wait for it… 😱 #...,2021-12-10 00:26:16,Taylor Red TV,2021-12-10 00:26:16,[None],22912715,0,0,1318,FALSE,TRUE,Travel & Events
Jeep® | The Middle,2021-02-06 23:58:17,Jeep,2021-02-06 23:58:17,jeep|jeep vehicle...,35708883,29814,18047,11565,FALSE,FALSE,Autos & Vehicles


## Elastic Search

In [31]:
# ES Hadoop

In [58]:
from elasticsearch import Elasticsearch

ModuleNotFoundError: No module named 'elasticsearch'